In [ ]:
import pandahouse
import pandas as pd
import numpy as np
from clickhouse_driver import Client
import plotly.graph_objects as go

In [ ]:
table = 'stroke'
database = 'prepare'
connection = {'host': 'http://localhost:8123', 
              'password':'1q2w3e4r', 
              'database': database}

client = Client('localhost', password = '1q2w3e4r')

In [ ]:
sql = """CREATE DATABASE IF NOT EXISTS {}""".format(database)
client.execute(sql)
client.execute("SHOW DATABASES")

In [ ]:
df_stroke = pandahouse.read_clickhouse('SELECT * FROM {}'.format(table), 
                                       index_col='uuid', 
                                       connection=connection)
df_stroke.head()

In [ ]:
columns = [
    "_50", "_100", "_150", "_200", "_250", "_300", "_350", "_400", "_450","_500",
    "_550", "_600", "_650", "_700", "_750", "_800", "_850", "_900", "_950", "_1000",
    "_1050", "_1100", "_1150", "_1200", "_1250", "_1300", "_1350", "_1400", "_1450", "_1500",
    "_1550", "_1600", "_1650", "_1700", "_1750", "_1800", "_1850", "_1900", "_1950", "_2000"
]
df_stroke[columns] = df_stroke[columns].round().astype('uint8')

In [ ]:
df_stroke.info()

In [ ]:
df_stroke.describe()

In [ ]:
for col in columns:
    df_stroke[col].replace(0, df_stroke.loc[df_stroke[col] != 0, col].median(), inplace=True)

df_stroke.describe()

In [ ]:
columns = ["mean_stroke_500",  "mean_stroke_1000", "mean_stroke_1500", "mean_stroke_2000"]
for i, col in enumerate(columns):
    df_stroke[col] = df_stroke.iloc[:, i*10:(i+1)*10].mean(axis=1).astype('uint8')

df_stroke

In [ ]:
client.execute("""DROP TABLE IF EXISTS {}.{} """.format(database, table))

sql = """
CREATE TABLE IF NOT EXISTS {}.{}
(
    uuid String,
    _50 UInt8,
    _100 UInt8,
    _150 UInt8,
    _200 UInt8,
    _250 UInt8,
    _300 UInt8,
    _350 UInt8,
    _400 UInt8,
    _450 UInt8,
    _500 UInt8,
    _550 UInt8,
    _600 UInt8,
    _650 UInt8,
    _700 UInt8,
    _750 UInt8,
    _800 UInt8,
    _850 UInt8,
    _900 UInt8,
    _950 UInt8,
    _1000 UInt8,
    _1050 UInt8,
    _1100 UInt8,
    _1150 UInt8,
    _1200 UInt8,
    _1250 UInt8,
    _1300 UInt8,
    _1350 UInt8,
    _1400 UInt8,
    _1450 UInt8,
    _1500 UInt8,
    _1550 UInt8,
    _1600 UInt8,
    _1650 UInt8,
    _1700 UInt8,
    _1750 UInt8,
    _1800 UInt8,
    _1850 UInt8,
    _1900 UInt8,
    _1950 UInt8,
    _2000 UInt8,
    mean_stroke_500 UInt8,
    mean_stroke_1000 UInt8,
    mean_stroke_1500 UInt8,
    mean_stroke_2000 UInt8,
    date_competition Date
) ENGINE = MergeTree(date_competition, (uuid), 8192)

""".format(database, table)

client.execute(sql)
client.execute("SHOW TABLES FROM {}".format(database))

In [ ]:
affected_rows = pandahouse.to_clickhouse(df_stroke, table=table, connection=connection)

---

## Чтение обработанной таблицы с темпами

---

In [ ]:
df = df_stroke

In [ ]:
df = pandahouse.read_clickhouse("""SELECT * FROM {}.{}""".format(database, table), 
                                index_col='uuid',
                                connection=connection)
df

In [ ]:
def iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    # Filtering Values between Q1-1.5IQR and Q3+1.5IQR
    return series[(Q1 - 1.5 * IQR <= series) & (series <= Q3 + 1.5 * IQR)]

In [ ]:
def plot_stroke_scatter(df):
    df = df.iloc[:1000]

    label_columns = [
        "_50", "_100", "_150", "_200", "_250", "_300", "_350", "_400", "_450","_500",
        "_550", "_600", "_650", "_700", "_750", "_800", "_850", "_900", "_950", "_1000",
        "_1050", "_1100", "_1150", "_1200", "_1250", "_1300", "_1350", "_1400", "_1450", "_1500",
        "_1550", "_1600", "_1650", "_1700", "_1750", "_1800", "_1850", "_1900", "_1950", "_2000"
    ]


    fig = go.Figure()

    for col in label_columns:
        filtered = iqr(df[col])
        fig.add_trace(
            go.Scatter(
                x=[col[1:]] * len(filtered),
                y=filtered,
                mode='markers',
                marker=dict(
                    color='rgba(255, 182, 193, .1)',
                    line_width=0.5
                ) 
            )
        )


    fig.show()

# plot_stroke_scatter(df)

In [ ]:
def plot_stroke_scatter(df):

    label_columns = [
        "_50", "_100", "_150", "_200", "_250", "_300", "_350", "_400", "_450","_500",
        "_550", "_600", "_650", "_700", "_750", "_800", "_850", "_900", "_950", "_1000",
        "_1050", "_1100", "_1150", "_1200", "_1250", "_1300", "_1350", "_1400", "_1450", "_1500",
        "_1550", "_1600", "_1650", "_1700", "_1750", "_1800", "_1850", "_1900", "_1950", "_2000"
    ]

    fig = go.Figure()

    for col in label_columns:
        filtered = iqr(df[col])
        size = filtered.value_counts(normalize=True, sort=False).sort_index()
        moda = size.idxmax()
        color = ['rgb(25, 70, 186, .7)'] * len(size)
        color[size.index.get_loc(moda)] = 'rgb(236, 11, 67)'

        fig.add_trace(
            go.Scatter(
                x=[col[1:]] * len(size),
                y=size.index,
                name=col[1:],
                mode='markers',
                    marker=dict(
                        size=size,
                        sizemode='area',
                        sizeref=2.*max(size)/(10.**2),
                        color=color
                    )
            )
        )

    fig.update_layout(showlegend=False, 
                      title="Наиболее используемые темпа")
    fig.show()

plot_stroke_scatter(df)

In [ ]:
def plot_stroke_scatter(df):
    label_columns = [
        "_50", "_100", "_150", "_200", "_250", "_300", "_350", "_400", "_450","_500",
        "_550", "_600", "_650", "_700", "_750", "_800", "_850", "_900", "_950", "_1000",
        "_1050", "_1100", "_1150", "_1200", "_1250", "_1300", "_1350", "_1400", "_1450", "_1500",
        "_1550", "_1600", "_1650", "_1700", "_1750", "_1800", "_1850", "_1900", "_1950", "_2000"
    ]
    df[label_columns] = df[label_columns][df[label_columns] >= df[label_columns].quantile(0.75)]

    fig = go.Figure()

    for col in label_columns[:10]:   
        fig.add_trace(
            go.Violin(
                x=[col[1:]] * len(df),
                y=df[col],
                name=col[1:],
                box_visible=True,
                meanline_visible=True
            )
        )

    fig.update_layout(showlegend=False, 
                      title="Наиболее используемые темпа")
    fig.show()

plot_stroke_scatter(df)

In [ ]:
fig = go.Figure()

traces = []
label_columns = [
    "_50", "_100", "_150", "_200", "_250", "_300", "_350", "_400", "_450","_500",
    "_550", "_600", "_650", "_700", "_750", "_800", "_850", "_900", "_950", "_1000",
    "_1050", "_1100", "_1150", "_1200", "_1250", "_1300", "_1350", "_1400", "_1450", "_1500",
    "_1550", "_1600", "_1650", "_1700", "_1750", "_1800", "_1850", "_1900", "_1950", "_2000"
]
for xd, yd in zip(label_columns[:10], df[label_columns[:10]].values):
        traces.append(go.Box(
            y=yd,
            name=xd[1:],
            boxpoints='outliers',
            jitter=0.5,
            whiskerwidth=0.2,
            marker=dict(
                size=2,
            ),
            line=dict(width=1),
        ))

layout = go.Layout(
    title='Points Scored by the Top 9 Scoring NBA Players in 2012',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig = go.Figure(data=traces, layout=layout)

fig.show()

In [ ]:
def plot_stroke_violin(df):
    label_columns = [
        "_50", "_100", "_150", "_200", "_250", "_300", "_350", "_400", "_450","_500",
        "_550", "_600", "_650", "_700", "_750", "_800", "_850", "_900", "_950", "_1000",
        "_1050", "_1100", "_1150", "_1200", "_1250", "_1300", "_1350", "_1400", "_1450", "_1500",
        "_1550", "_1600", "_1650", "_1700", "_1750", "_1800", "_1850", "_1900", "_1950", "_2000"
    ]

    fig = go.Figure()
    
    for col in label_columns:
        filtered = iqr(df[col])
        
        fig.add_trace(
            go.Violin(
                x=[col[1:]] * len(filtered),
                y=filtered,
                name=col[1:],
                box_visible=True,
            )
        )

    fig.update_layout(showlegend=False, 
                      title="Наиболее используемые темпа",     
                      xaxis_title="Дистанция (м)",
                      yaxis_title="Темп (1/мин)",)
    fig.show()

plot_stroke_violin(df)

In [ ]:
def plot_heatmap_stroke(df, split=1):
    label_columns = [
        "_50", "_100", "_150", "_200", "_250", "_300", "_350", "_400", "_450","_500",
        "_550", "_600", "_650", "_700", "_750", "_800", "_850", "_900", "_950", "_1000",
        "_1050", "_1100", "_1150", "_1200", "_1250", "_1300", "_1350", "_1400", "_1450", "_1500",
        "_1550", "_1600", "_1650", "_1700", "_1750", "_1800", "_1850", "_1900", "_1950", "_2000"
    ]
    
    filtered = iqr(df[label_columns])
    label = [col[1:] for col in label_columns]
    
    if split != 1:
        split_df = pd.DataFrame()
        columns = [arr.tolist() for arr in np.array_split(label_columns, split)]
        for cols in columns:
            split_df[cols[-1][1:]] = filtered[cols].mean(axis=1)
        filtered = split_df
        
        step = len(label) // split
        label = label[step-1::step]
        
    

    fig = go.Figure(data=go.Heatmap(x=label,
                                    y=label,
                                    z=filtered.corr(),
                                    colorscale = "YlOrRd"))

    fig.show()
    
plot_heatmap_stroke(df, split=20)

In [ ]:
df[label_columns].corr()

In [ ]:
df_stroke.info()

In [ ]:
df_stroke.describe()

In [ ]:
label_columns = [
    "_50", "_100", "_150", "_200", "_250", "_300", "_350", "_400", "_450","_500",
    "_550", "_600", "_650", "_700", "_750", "_800", "_850", "_900", "_950", "_1000",
    "_1050", "_1100", "_1150", "_1200", "_1250", "_1300", "_1350", "_1400", "_1450", "_1500",
    "_1550", "_1600", "_1650", "_1700", "_1750", "_1800", "_1850", "_1900", "_1950", "_2000"
]
iqr(df_stroke[label_columns]).describe()

In [ ]:
label_columns = [
    "_50", "_100", "_150", "_200", "_250", "_300", "_350", "_400", "_450","_500",
    "_550", "_600", "_650", "_700", "_750", "_800", "_850", "_900", "_950", "_1000",
    "_1050", "_1100", "_1150", "_1200", "_1250", "_1300", "_1350", "_1400", "_1450", "_1500",
    "_1550", "_1600", "_1650", "_1700", "_1750", "_1800", "_1850", "_1900", "_1950", "_2000"
]

np.array_split(label_columns, 4)